<a href="https://colab.research.google.com/github/mullerino/Topicos-em-IA/blob/main/Processamento_de_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alunos: João Arthur e Leandro Müller

Relatório de Status: Classificação de Mensagens de Texto como Spam

**Projeto:** Classificação de Mensagens de Texto como Spam  
**Fonte dos Dados:** Kaggle - Spam Text Message Classification  
**Objetivo:** Detectar mensagens de texto classificadas como spam utilizando três abordagens diferentes de modelagem.

---

## 1. Modelos Implementados

Foram implementados três modelos diferentes para a tarefa de classificação de mensagens de texto como spam:

### 1.1 Logistic Regression
- **Pré-processamento:** Foi utilizado o método TF-IDF (`TfidfVectorizer`) para transformar o texto em vetores numéricos.
- **Modelo:** Foi treinado um modelo de Regressão Logística (`LogisticRegression`) ajustado para lidar com o desequilíbrio das classes.
- **Resultados:**
  - Precision, Recall e F1-Score foram calculados, e as métricas indicam que o modelo é simples e eficiente para tarefas de classificação de texto, mas pode ter limitações dependendo do desequilíbrio das classes.

### 1.2 Langchain com DistilBERT
- **Modelo:** O modelo DistilBERT foi integrado usando o framework Langchain, permitindo fine-tuning em mensagens de texto.
- **Tokenização e Treinamento:** O modelo foi treinado usando `Trainer` da Hugging Face para tarefas de classificação.
- **Resultados:**
  - As métricas de avaliação (Precision, Recall, F1-Score) indicam um desempenho equilibrado entre eficiência e precisão, sendo uma escolha sólida para tarefas de NLP em tempo real.

### 1.3 LLaMA (ou Modelo Alternativo)
- **Modelo:** Inicialmente planejado para usar o modelo LLaMA, enfrentamos dificuldades de acesso. Como alternativa, foi utilizado o modelo GPT-Neo (`EleutherAI/gpt-neo-2.7B`) devido à sua disponibilidade pública.
- **Tokenização e Treinamento:** O modelo foi treinado seguindo um pipeline semelhante ao do DistilBERT, mas com ajustes para acomodar as características específicas do GPT-Neo.
- **Resultados:**
  - O modelo GPT-Neo apresentou bom desempenho nas métricas de avaliação, com potencial para captura de nuances mais complexas em texto.

---

## 2. Desafios e Soluções Implementadas

### 2.1 Desequilíbrio de Classes
- **Desafio:** O modelo de Regressão Logística inicial teve dificuldade em identificar a classe "spam" devido ao desequilíbrio.
- **Solução:** Ajustes foram feitos no `class_weight` do modelo e no threshold de decisão, resultando em métricas de precisão, recall e F1-Score mais equilibradas.

### 2.2 Acesso ao Modelo LLaMA
- **Desafio:** O modelo LLaMA original não estava disponível no repositório público da Hugging Face.
- **Solução:** Como solução alternativa, utilizamos o modelo GPT-Neo, que está disponível publicamente e possui características semelhantes em termos de poder computacional.

### 2.3 Treinamento em Grande Escala
- **Desafio:** O treinamento de modelos grandes como DistilBERT e GPT-Neo pode ser computacionalmente intenso.
- **Solução:** Foram utilizados pequenos subconjuntos dos dados e configurações otimizadas (como batch size e número de épocas) para acelerar o treinamento sem comprometer significativamente o desempenho.

---

## 3. Próximos Passos

- **Avaliação e Comparação Detalhada:** Realizar uma comparação detalhada das métricas entre os três modelos para identificar o melhor desempenho para a tarefa específica.
- **Refinamento dos Modelos:** Considerar técnicas de ensemble para combinar os modelos e melhorar a robustez e a precisão da classificação.
- **Desdobramento para Produção:** Planejar o desdobramento do modelo escolhido em um ambiente de produção, integrando com sistemas de alerta para detecção de spam em tempo real.

---

**Conclusão:**

Estamos em um estágio avançado do projeto, com três abordagens distintas implementadas e avaliadas. Cada modelo apresenta suas próprias vantagens, e a escolha final dependerá do balanceamento entre desempenho e recursos computacionais disponíveis. As próximas etapas se concentrarão na avaliação comparativa e na preparação para a implantação.

## Coleta e Pré-processamento de Dados

In [ ]:
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

# 1. Carregar o dataset
df = pd.read_csv("spam.csv", encoding='latin-1')

# 2. Manter apenas as colunas necessárias e renomeá-las
df = df[['Category', 'Message']]
df.columns = ['label', 'text']

# Usar apenas uma amostra dos dados para treinamento rápido
df_sample = df.sample(frac=0.1, random_state=42)

# 3. Função de limpeza do texto
def preprocess_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remover pontuação
    text = re.sub("\s+", " ", text)  # Remover espaços em branco extras
    return text

df_sample['text'] = df_sample['text'].apply(preprocess_text)

# 4. Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_sample['text'], df_sample['label'], test_size=0.3, random_state=42)


## Langchain com DistilBERT

In [ ]:
# 5. Tokenização dos dados
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)

# 6. Criar dataset personalizado
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(1 if self.labels.iloc[idx] == "spam" else 0)  # Convertendo labels para binário
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SpamDataset(train_encodings, y_train)
test_dataset = SpamDataset(test_encodings, y_test)

# 7. Configurar e treinar o modelo
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Apenas 1 época
    per_device_train_batch_size=8,  # Batch menor para acelerar
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()
trainer.evaluate()

## Logistic Regression



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. Pré-processamento usando TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 2. Treinamento com Logistic Regression, ajustando class_weight
log_reg_model = LogisticRegression(class_weight='balanced')
log_reg_model.fit(X_train_tfidf, y_train)

# 3. Ajuste do threshold de decisão e previsão
y_prob = log_reg_model.predict_proba(X_test_tfidf)[:, 1]
y_pred = ['spam' if prob > 0.3 else 'ham' for prob in y_prob]  # Threshold ajustado

# 4. Avaliação do modelo
precision = precision_score(y_test, y_pred, pos_label='spam')
recall = recall_score(y_test, y_pred, pos_label='spam')
f1 = f1_score(y_test, y_pred, pos_label='spam')

print(f"Logistic Regression - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")



Logistic Regression - Precision: 0.3571, Recall: 0.9615, F1-Score: 0.5208


## LLaMA

In [ ]:
from transformers import GPTNeoForSequenceClassification, GPT2Tokenizer, Trainer, TrainingArguments
import torch

# Carregando o modelo e o tokenizer GPT-Neo
tokenizer_gpt = GPT2Tokenizer.from_pretrained('EleutherAI/gpt-neo-2.7B')
model_gpt = GPTNeoForSequenceClassification.from_pretrained('EleutherAI/gpt-neo-2.7B', num_labels=2)

# Tokenização
train_encodings_gpt = tokenizer_gpt(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings_gpt = tokenizer_gpt(list(X_test), truncation=True, padding=True, max_length=128)

# Criar dataset personalizado
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(1 if self.labels.iloc[idx] == "spam" else 0)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset_gpt = SpamDataset(train_encodings_gpt, y_train)
test_dataset_gpt = SpamDataset(test_encodings_gpt, y_test)

# Configuração do treinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer_gpt = Trainer(
    model=model_gpt,
    args=training_args,
    train_dataset=train_dataset_gpt,
    eval_dataset=test_dataset_gpt
)

# Treinamento do modelo GPT-Neo
trainer_gpt.train()

# Avaliação do modelo GPT-Neo
results_gpt = trainer_gpt.evaluate()

# Cálculo das métricas Precision, Recall e F1-Score
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred_gpt = trainer_gpt.predict(test_dataset_gpt).predictions.argmax(-1)

precision = precision_score(y_test, y_pred_gpt, pos_label=1)
recall = recall_score(y_test, y_pred_gpt, pos_label=1)
f1 = f1_score(y_test, y_pred_gpt, pos_label=1)

print(f"GPT-Neo - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-f5aced0377f7>", line 6, in <cell line: 6>
    model_gpt = GPTNeoForSequenceClassification.from_pretrained('EleutherAI/gpt-neo-2.7B', num_labels=2)
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3838, in from_pretrained
    ) = cls._load_pretrained_model(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 4246, in _load_pretrained_model
    error_msgs = _load_state_dict_into_model(model_to_load, state_dict, start_prefix)
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 711, in _load_state_dict_into_model
    load(model_to_load, state_dict, prefix=start_prefix)
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 709, in load
    

TypeError: object of type 'NoneType' has no len()